In [ ]:
!pip install unidecode 

In [ ]:
import pandas as pd
import re
from unidecode import unidecode
import pickle

In [ ]:
data = pd.read_csv('/content/fire_tv_telecommande_vocale.csv')
data

,Commentaire,Label
0,Le contenu de la boite est identique à l’ancie...,5
1,Une petite merveille pour moins de 30€ !Instal...,5
2,J en ai fait l acquisition sans être vraiment ...,5
3,Pas mal de bugs quand même du fait qu’on ne pu...,3
4,"Fan des appareils Amazon, j'ai voulu acheter l...",5
...,...,...
1824,👍,5
1825,Je l’avais commander sur Boulanger. Cette clé ...,5
1826,"Produit facile à installer, très pratique qua...",5
1827,Simple et pratique bon rapport qualite prix,5


In [ ]:
print(data.shape)

(1829, 2)


#Nettoyages du Data

##Supprission des commentaires répétés

In [ ]:
data.drop_duplicates(subset=['Commentaire'],inplace=False)
data.shape

(1829, 2)

## Enlever les textes < seuil_mot

In [ ]:
seuil_mot=15
def nb_mot(commentaire):
    return len(commentaire.split(' '))

data['Longueur']=data['Commentaire'].apply(nb_mot)
data=data[data['Longueur']>seuil_mot]

In [ ]:
data

,Commentaire,Label,Longueur
0,Le contenu de la boite est identique à l’ancie...,5,445
1,Une petite merveille pour moins de 30€ !Instal...,5,129
2,J en ai fait l acquisition sans être vraiment ...,5,158
3,Pas mal de bugs quand même du fait qu’on ne pu...,3,95
4,"Fan des appareils Amazon, j'ai voulu acheter l...",5,667
...,...,...,...
1812,Nous utilisons ce produit pour la réception de...,4,20
1818,Vraiment très bien je recommande la preuve va...,5,17
1825,Je l’avais commander sur Boulanger. Cette clé ...,5,36
1826,"Produit facile à installer, très pratique qua...",5,35


## Binarisation du Label
### Label = 1,2,3 ==> 0
### Label = 4,5 ==> 1

In [ ]:
data.loc[data[data['Label'].astype(int)<4].index,'Label']=0
data.loc[data[data['Label'].astype(int)>3].index,'Label']=1
data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Commentaire,Label,Longueur
0,Le contenu de la boite est identique à l’ancie...,1,445
1,Une petite merveille pour moins de 30€ !Instal...,1,129
2,J en ai fait l acquisition sans être vraiment ...,1,158
3,Pas mal de bugs quand même du fait qu’on ne pu...,0,95
4,"Fan des appareils Amazon, j'ai voulu acheter l...",1,667
...,...,...,...
1812,Nous utilisons ce produit pour la réception de...,1,20
1818,Vraiment très bien je recommande la preuve va...,1,17
1825,Je l’avais commander sur Boulanger. Cette clé ...,1,36
1826,"Produit facile à installer, très pratique qua...",1,35


In [ ]:
Comments['Label'].value_counts()

1    127
0    127
Name: Label, dtype: int64

In [ ]:
Comments

,Commentaire,Label,Longueur
805,"Ce stick amazon est une réussite, in branche s...",1,40
165,acheté en promo à 20 € pour une ancienne tv av...,1,63
547,"C'est juste magique, nous en avons commandé 3 ...",1,31
962,Installation facile - privilégiez le cable d'a...,1,36
102,Your browser does not support HTML5 video.\n\n...,1,120
...,...,...,...
1106,bon produit dans le principe MAIS il arrive TR...,0,20
1778,Bonjour.faut il une tele connectée pour que ça...,0,32
276,Attention il y a souvent de désagréable perte ...,0,37
370,"Plante souvent, impossible à éteindre, du coup...",0,28


#### 3) Nettoyage spécifique pour de l’analyse de texte

- Stop words
- carcteres speciaux (dont ponctuation)
- normalisation (accents et minuscule)

In [ ]:
def nettoyage_avec_lem(texte):
    lem=pickle.load(open('/content/sample_data/lemmatize_fr.pickle','rb'))
    
    # la liste de stop words
    with open('/content/sample_data/stop-w_fr.txt') as f:
        s_w=f.read().split('\n')

    tmp=[]
    
    # Normalisation : accents et minuscule
    texte=unidecode(texte.lower())
    p='[a-z]{2,}'
    for mot in re.findall(p,texte):
        if mot in s_w:
            continue
        else:
            tmp.append(mot)
    return ' '.join(tmp)

In [ ]:
negatif=[nettoyage_avec_lem(elem) for elem in Comments[Comments['Label']==0]['Commentaire']]
positif=[nettoyage_avec_lem(elem) for elem in Comments[Comments['Label']==1]['Commentaire']]

In [ ]:
negatif

['produit vraie arnaque telecharger netflix faut imperativement carte bancaire enregistree amazon sachant amazon fiable cartes bleus paye frais acheter apple tv adresse mail connecte',
 'achete produit mai vacances fin septembre apercu neconnecte tv appele service vente amazon informer technicien quise prenomme ghiane propose remplacer gratuitement donne serie chiffrespour permettre renvoyer materiel defectueux attend rachete mecontent service vente ca reflete image faisais amazon',
 'perte telecommande modele tele sony telecommande universelle chere naviguer clef appeler alicia pratique femme teste miracle amazon prime bonne telecommande regarde ca marche ok',
 'deuxieme televiseur payant acceder chaines normal',
 'mal vraiment pratique facile utilisation certe appareil plante application twitch comble sachant amazon detient plate forme bref petit gadget pratique faillible',
 'redire qualite produit fire cube fire tv stick normal achete offrir etoiles enleve livraison boite ecrasee de

In [ ]:
positif

['stick amazon reussite in branche hdmi met code wifi maison parti qualite image bonne interface jeu enfant',
 'achete promo ancienne tv port hdmi job jeter tv tv cuisine repris parametres codes applis officielles installees installer kodi cuisine top',
 'magique commande tele connecte capricieuse top bug facile installation appli telecommande parfait',
 'installation facile privilegiez cable alimentation port usb tv stable tele telechargement rapide fibre craquer',
 'your browser does not support html video parfait chambre garcons termes streaming reactif niveau latence bugs constates qualite video hd bout bout choisi fire stick lite commande tv chambre enfants commande vocale dicter souhaits tv charge volume allumer eteindre parents faisons chose dispose fire stick telecommande salon indispensable piece vivre recommande produit yeux fermes hesitez espere commentaire eclairci hesitent',
 'cle vraiment super offre chose apple tv facile mettre marche configurer retrouve min tele piece a

In [4]:
# !apt-get install texlive texlive-xetex texlive-latex-extra pandoc
# !pip install pypandoc

In [3]:
!cp '/content/drive/MyDrive/Colab Notebooks/Text_Data_Cleaning.ipynb' ./

In [5]:
!jupyter nbconvert --to PDF '/content/drive/MyDrive/Colab Notebooks/Text_Data_Cleaning.ipynb'


[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Text_Data_Cleaning.ipynb to PDF
[NbConvertApp] Writing 95564 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 77258 bytes to /content/drive/MyDrive/Colab Notebooks/Text_Data_Cleaning.pdf
